basic mwalib correlator context

In [ ]:
:dep mwalib = {version = ">=0.9.4"}
:dep mwa_rust_core = {version = "0.1.1"}

extern crate mwalib;

use mwa_rust_core::mwalib::{
    CorrelatorContext,
    MwalibError
}

let metafits_path = "1254670392.metafits";
let fits_files = ["1254670392_20191009153257_gpubox01_00.fits"];
let mut context: CorrelatorContext = CorrelatorContext::new(&metafits_path, &fits_files)
            .expect("unable to get mwalib context");

println!("mwalib correlator context:\n{}", &context);

reading visibility data

In [ ]:
:dep birli = {version = "0.1.8"}
:dep ndarray = {version = ">=0.14,<0.16", features = ["rayon"]}
:dep mwa_rust_core = {version = ">=0.1.1"}
:dep mwalib = {version = ">=0.9.4"}

// extern crate mwa_rust_core;
extern crate mwalib;

use birli::{
    context_to_jones_array,
    get_flaggable_timesteps,
};
use mwa_rust_core::{
    Jones,
    mwalib::{
        CorrelatorContext,
        MwalibError
    }
};

let metafits_path = "1254670392.metafits";
let fits_files = ["1254670392_20191009153257_gpubox01_00.fits"];
let context = CorrelatorContext::new(&metafits_path, &fits_files)
            .expect("unable to get mwalib context");

println!("mwalib correlator context:\n{}", &context);

let common_coarse_chan_idxs = context.common_coarse_chan_indices.clone();
let flaggable_timestep_idxs =
    get_flaggable_timesteps(&context).expect("unable to determine flaggable timesteps");

let mwalib_coarse_chan_range =
    *common_coarse_chan_idxs.first().unwrap()..(*common_coarse_chan_idxs.last().unwrap() + 1);
println!("mwalib_coarse_chan_range: {:?}", mwalib_coarse_chan_range);
let mwalib_timestep_range =
    *flaggable_timestep_idxs.first().unwrap()..(*flaggable_timestep_idxs.last().unwrap() + 1);
println!("mwalib_timestep_range: {:?}", mwalib_timestep_range);

let (mut jones_array, _) = context_to_jones_array(
    &context,
    &mwalib_timestep_range,
    &mwalib_coarse_chan_range,
    None,
);

println!("jones array dimensions: {:?}", jones_array.dim());

plot visibilities

In [ ]:
:dep plotly = {version = ">=0.6.0", features = ["kaleido"]}
:dep itertools-num = {version = "0.1.3"}
:dep ndarray = {version = ">=0.14,<0.16", features = ["rayon"]}
:dep birli = {version = "0.1.8"}
:dep mwa_rust_core = {version = ">=0.1.1"}
:dep mwalib = {version = ">=0.9.4"}

extern crate mwalib;
extern crate plotly;
extern crate rand_distr;
extern crate itertools_num;
extern crate itertools;

use birli::{
    context_to_jones_array,
    get_flaggable_timesteps,
};
use mwa_rust_core::{
    Jones,
    mwalib::{
        CorrelatorContext,
        MwalibError
    }
};

use itertools_num::linspace;
use plotly::common::{
    ColorScale, ColorScalePalette, DashType, Fill, Font, Line, LineShape, Marker, Mode, Title,
};
use plotly::layout::{Axis, BarMode, Layout, Legend, TicksDirection};
use plotly::{Bar, NamedColor, Plot, Rgb, Rgba, Scatter, ImageFormat};
use rand_distr::{Distribution, Normal, Uniform};

use ndarray::{ArrayBase, OwnedRepr, Dim, Axis};

let metafits_path = "1254670392.metafits";
let fits_files = ["1254670392_20191009153257_gpubox01_00.fits"];
let context = CorrelatorContext::new(&metafits_path, &fits_files)
            .expect("unable to get mwalib context");
// println!("mwalib correlator context:\n{}", &context);

let common_coarse_chan_idxs = context.common_coarse_chan_indices.clone();
let flaggable_timestep_idxs =
    get_flaggable_timesteps(&context).expect("unable to determine flaggable timesteps");

let mwalib_coarse_chan_range =
    *common_coarse_chan_idxs.first().unwrap()..(*common_coarse_chan_idxs.last().unwrap() + 1);
// println!("mwalib_coarse_chan_range: {:?}", mwalib_coarse_chan_range);
let mwalib_timestep_range =
    *flaggable_timestep_idxs.first().unwrap()..(*flaggable_timestep_idxs.last().unwrap() + 1);
// println!("mwalib_timestep_range: {:?}", mwalib_timestep_range);

let (mut jones_array, _) = context_to_jones_array(
    &context,
    &mwalib_timestep_range,
    &mwalib_coarse_chan_range,
    None,
);

println!("jones array dimensions: {:?}", jones_array.dim());


// let's plot a particular slice of visibilities
let timestep_idx = mwalib_timestep_range.start;
let baseline_idx = 300; 

// our vector of frequencies (the X axis)
let f = context.get_fine_chan_freqs_hz_array(&mwalib_coarse_chan_range.clone().collect::<Vec<_>>());

// info about the slice we're plotting
println!("looking at slice:");
println!(" -> timestep {:?}", context.timesteps[timestep_idx], );
println!(" -> frequency range {} Hz - {} Hz", f.first().unwrap(), &f.last().unwrap() );
let baseline = context.metafits_context.baselines[baseline_idx].clone();
println!(" -> baseline {} ( ant {} vs {} )", &baseline_idx, &baseline.ant1_index, &baseline.ant2_index );
let pol_order = ["xx", "xy", "yx", "yy"];

// The jones array, with only the selected baseline (Axis 2) and timestep (Axis 0)
let jones_plot_view = jones_array.index_axis_move(Axis(2), baseline_idx).index_axis_move(Axis(0), timestep_idx);

// start with our plot object
let mut plot = Plot::new();
// for each polarization, add a trace to the plot.
for (pol_idx, pol_name) in pol_order.iter().enumerate() {
    // our vector of complex magnidues within the selected slice of visibilities.
    let r = jones_plot_view.iter().map(|jones| {
        jones[pol_idx].norm()
    }).collect::<Vec<_>>();
    
    let trace = Scatter::new(f.clone(), r).mode(Mode::LinesMarkers).name(&format!("ts={}, pol={}", &timestep_idx, &pol_name).as_str());
    plot.add_trace(trace);
}


let layout = Layout::new().height(800);
plot.set_layout(layout);
plot.notebook_display();